Final Project - AWS ETL Data Streaming 
We decided to create a Data Pipeline on AWS that could facilitate the process of building a Natural Language Processing tool. The tool will utilize a simple machine learning algorithm or one/two layer neural network to train a topic modeler or sentiment analyzer or a recommender. The technologies (tentative) that will be used in this project

* Terraform to create template for cloud managment and configuration 
* Lambda and SAM to streamline a simple ETL data pipeline that extracts, processes, and load the data from Twitter API(tentative)
* Eventbridge(Cloudwatch events) to log and monitor state changes 
* S3 Bucket to store the raw retrieved data (potentially utilize DynamoDB to store the parsed and processed data for future analytics)
* Sagemaker to create a notebook for hosting the ML project code, fine-tuning  parameters, pipelining and storing the models
* Pandas to ingest and wrangle the data and preprocess it for ML training
* EC2 to create a VM that hosts the workflow or provide workers for training if needed 
* (Tentative) DynamoDB to store a structured version of the output data for future reference 

Terraform templapte 
to create  and manage an aws infrastructure 


Run test code for Terraform templates

In [ ]:
import os
os.chdir('/Users/shiyishen/nlp/aws-data-engineering/aws-data-pipeline/cdk-tf/')


### Install dependencies and packages for the project

In [ ]:
!pip install boto3
!pip install awswrangler
!pip install tweepy
!pip install numpy


### Data streaming with Lambda from Twitter API

In [2]:

import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("AAAAAAAAAAAAAAAAAAAAADJKhwEAAAAA%2BkjQU9%2BBJPu%2FX5QrarVf9ppyAfE%3D99qn5c6SvhI9vegVNJax3Lo4jRqZWWqOYzsMNrTVHQNDO8CqNU")


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "election place_country:US"},
        {"value": "political"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent=4, sort_keys=True))


def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)


if __name__ == "__main__":
    main()

Exception: Cannot get rules (HTTP 401): {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}

In [ ]:
# TODO
# Twitter pulling class 

In [ ]:
# Twitter processing class 


In [ ]:
# Twitter loading class 

### Data Wrangling and transformation with Pandas

### Data loading to S3

### Sagemaker Machine Learning LDA Algorithm on EC2

In [ ]:
#data loading

In [ ]:
#sagemaker imports 
#